# Molecular Distance Geometry

## Basic Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy
from numba import jit, njit
%matplotlib inline

In [2]:
def dist(p, q):
    return np.linalg.norm(p - q)


def norm(v):
    return np.linalg.norm(v)

## Reading the input from a csv

In [25]:
data = pd.read_csv("7_18.csv")

In [26]:
data

,u,v,duv
0,5,6,1.526000
1,2,6,3.949161
2,4,6,2.491390
3,1,4,3.808751
4,4,7,2.666295
5,3,5,2.491390
6,2,4,2.491390
7,1,5,4.615439
8,3,4,1.526000
9,2,5,3.561412


## Converting the data to a graph

In [27]:
n = max(data['v'])
m = len(data['duv'])

In [28]:
n, m

(7, 18)

In [29]:
g = [[] for _ in range(n + 1)]
edges = []
prune_edges = []

for i, r in data.iterrows():
    u = int(r['u'])
    v = int(r['v'])
    duv = r['duv']

    g[u].append((v, duv))
    g[v].append((u, duv))
    edges.append((u, v, duv))

    if u + 3 < v:
        prune_edges.append((u, v))

## Symetries

In [30]:
def sym(n, prune_edges):
    w = [0 for _ in range(n + 1)]

    for ep in prune_edges:
        u = ep[0]
        v = ep[1]

        w[u] = max(w[u], v)

    eps = []

    for i in range(n + 1):
        if w[i] != 0:
            eps.append((i, w[i]))

    u_bar, v_bar = eps[0]
    S_bar = []

    for k in range(1, len(eps)):
        u_k = eps[k][0]
        v_k = eps[k][1]

        if v_bar >= u_k + 3:
            v_bar = max(v_bar, v_k)
        else:
            S_bar.append((u_bar + 4, v_bar))
            u_bar = u_k
            v_bar = v_k

    S_bar.append((u_bar + 4, v_bar))

    S = [4]
    i = 0
    for v in range(5, n + 1):
        u_bar = S_bar[i][0]
        v_bar = S_bar[i][1]

        if not (u_bar <= v and v <= v_bar):
            S.append(v)

        if v == v_bar and i + 1 < len(S_bar):
            i += 1

    return np.array(S)

In [31]:
sym(n, prune_edges)

array([4, 7])

## Branch & Prune

In [32]:
def ComprimentoAresta(g, u, v):
    duv = 0.0

    for i in range(len(g[u])):
        if g[u][i][0] == v:
            duv = g[u][i][1]
            break

    return duv


def PontosIniciais(n, g):
    X = np.array([np.array([0.0, 0.0, 0.0]) for _ in range(n + 1)])

    d12 = ComprimentoAresta(g, 1, 2)
    d13 = ComprimentoAresta(g, 1, 3)
    d23 = ComprimentoAresta(g, 2, 3)

    cth = (d12**2 + d23**2 - d13**2) / (2 * d12 * d23)
    th = np.arccos(cth)
    sth = np.sin(th)

    X[2] = np.array([-d12, 0.0, 0.0])
    X[3] = np.array([-d12 + d23 * cth, d23 * sth, 0.0])

    return X

In [33]:
def Intersecao3Esferas(A, ra, B, rb, C, rc):
    x1 = A
    x2 = B
    x3 = C
    #     print("Intersecao3Esferas ", x1, x2, x3)
    d12 = dist(x1, x2)
    d13 = dist(x1, x3)
    d14 = ra
    d23 = dist(x2, x3)
    d24 = rb
    d34 = rc

    r = (x3 - x2) / d23
    k234 = (d24**2 + d23**2 - d34**2) / (2 * d23)
    p234 = np.sqrt(d24**2 - k234**2)

    v = x1 - x2
    k123 = (d12**2 + d23**2 - d13**2) / (2 * d23)
    p123 = np.sqrt(d12**2 - k123**2)

    alpha = ((k234 - k123)**2 + p234**2 + p123**2) / (2 * p234 * p123)
    beta = -(1 / (2 * p234 * p123))

    cw = alpha + beta * d14**2
    sw = np.sqrt(1 - cw**2)

    X1 = x2 + k234 * r + (p234 / p123) * (
        (v - k123 * r) * cw + np.cross(r, v) * sw)
    X2 = x2 + k234 * r + (p234 / p123) * (
        (v - k123 * r) * cw - np.cross(r, v) * sw)

    return np.array([X1, X2])

In [34]:
def CalculaPonto(g, X, b, k):
    d1 = ComprimentoAresta(g, k - 3, k)
    d2 = ComprimentoAresta(g, k - 2, k)
    d3 = ComprimentoAresta(g, k - 1, k)
    #     print("CalculaPonto ", X[k-3], d1, X[k-2], d2, X[k-1], d3)
    pp = Intersecao3Esferas(X[k - 3], d1, X[k - 2], d2, X[k - 1], d3)
    return pp[b]


def CalculaErro(g, X, v):
    erro = 0.0
    for i in range(len(g[v])):
        u = g[v][i][0]
        di = g[v][i][1]

        if u < v:
            duv = dist(X[u], X[v])
            erro += (duv**2 - di**2)**2

    return np.sqrt(erro)

In [35]:
def bp(n, g, onesol=False, delta=10**-6):
    X = PontosIniciais(n, g)
    branch = np.zeros(n + 1, dtype=int)
    vist = np.zeros(n + 1, dtype=int)
    vist[1] = 1
    vist[2] = 1
    vist[3] = 1

    sol = {"X": [], "branch": []}

    k = 4

    while k > 3:
        vist[k] += 1
        X[k] = CalculaPonto(g, X, branch[k], k)
        erro = CalculaErro(g, X, k)

        prune = erro > delta

        if not prune and k == n:
            sol["X"].append(np.copy(X))
            sol["branch"].append(np.copy(branch))
            if onesol:
                return sol

        prune = prune or k == n

        if prune and branch[k] == 1:
            while k > 3 and branch[k] == 1:
                k -= 1

        if prune and branch[k] == 0:
            branch[k] = 1
            for i in range(k + 1, n + 1):
                branch[i] = 0

        if not prune:
            k += 1

    return sol

In [36]:
sol = bp(n, g)

## Sym BP

In [37]:
def ReflexaoPlano(x1, x2, x3, x):
    v1 = x1 - x2
    v2 = x3 - x2
    n = np.cross(v1, v2)
    n = n / norm(n)
    t = 2 * np.dot(n, x2 - x)
    return x + t * n


def TrocaSinal(s, piv):
    x1 = s["X"][piv - 3]
    x2 = s["X"][piv - 2]
    x3 = s["X"][piv - 1]
    t = copy.deepcopy(s)

    for i in range(piv, len(t['X'])):
        t['X'][i] = ReflexaoPlano(x1, x2, x3, t['X'][i])
        t['branch'][i] = 1 - t['branch'][i]

    return t

In [38]:
def EncontraCaminhos(sol, S):
    tree = [0 for _ in range(2**(len(S) + 1))]
    tree[1] = copy.deepcopy(sol)

    piv = 0
    k = 0
    # print("tree[1]:", tree[1] )
    for i in range(1, 2**len(S)):
        if 2**k == i:
            piv = S[k]
            k += 1
        tree[2 * i] = copy.deepcopy(tree[i])
        tree[2 * i + 1] = copy.deepcopy(TrocaSinal(tree[i], piv))

    return tree[2**len(S):2**(len(S) + 1)]

In [39]:
def symbp(n, g, prune_edges):
    sol = bp(n, g, True)
    S = sym(n, prune_edges)
    first_sol = {"X": sol["X"][0], "branch": sol["branch"][0]}
    sol = {"X": [], "branch": []}
    ans = EncontraCaminhos(first_sol, S)

    for a in ans:
        sol["X"].append(a["X"])
        sol["branch"].append(a["branch"])

    return sol

In [40]:
sol = symbp(n, g, prune_edges)

## Split

In [41]:
def SplitInstance(n, g, S):
    S = np.append(S, [n + 1])

    h = []

    for i in range(1, len(S)):
        u = S[i - 1] - 3
        v = S[i]

        it = {
            "n": v - u,
            "m": 0,
            "edges": [],
            "g": [[] for _ in range(v - u + 1)]
        }

        for j in range(u, v):
            for k, d in g[j]:
                if j < k and k < v:
                    it["edges"].append((j - u + 1, k - u + 1, d))
                    it["g"][j - u + 1].append((k - u + 1, d))
                    it["g"][k - u + 1].append((j - u + 1, d))
                    it["m"] += 1

        h.append(it)

    return h

In [42]:
def EncontraSolucao(n, g, prune_edges, S):
    h = SplitInstance(n, g, S.copy())

    branch = [0, 0, 0, 0]

    for it in h:
        sol = bp(it['n'], it['g'], True)
        bb = sol["branch"][0][4:it['n'] + 1]
        for b in bb:
            branch.append(b)
    branch = np.array(branch)

    X = PontosIniciais(n, g)
    for k in range(4, n + 1):
        X[k] = CalculaPonto(g, X, branch[k], k)

    return {"X": [X], "branch": [branch]}


def splitbp(n, g, prune_edges):
    S = sym(n, prune_edges)
    sol = EncontraSolucao(n, g, prune_edges, S)
    first_sol = {"X": sol["X"][0], "branch": sol["branch"][0]}
    sol = {"X": [], "branch": []}
    ans = EncontraCaminhos(first_sol, S)

    for a in ans:
        sol["X"].append(a["X"])
        sol["branch"].append(a["branch"])

    return sol

In [43]:
sol = splitbp(n, g, prune_edges)

In [44]:
X = sol["X"]
branch = sol["branch"]

In [45]:
branch

[array([0, 0, 0, 0, 0, 1, 1, 0]),
 array([0, 0, 0, 0, 0, 1, 1, 1]),
 array([0, 0, 0, 0, 1, 0, 0, 1]),
 array([0, 0, 0, 0, 1, 0, 0, 0])]

In [46]:
for X in sol['X']:
    print(X[1:])

[[ 0.          0.          0.        ]
 [-1.526       0.          0.        ]
 [-2.03375551  1.43904842  0.        ]
 [-3.48238364  1.4663484  -0.47896477]
 [-3.5869252   2.33224155 -1.7311533 ]
 [-2.52933762  3.43105962 -1.67839585]
 [-1.29164169  2.91015495 -0.9535213 ]]
[[ 0.          0.          0.        ]
 [-1.526       0.          0.        ]
 [-2.03375551  1.43904842  0.        ]
 [-3.48238364  1.4663484  -0.47896477]
 [-3.5869252   2.33224155 -1.7311533 ]
 [-2.52933762  3.43105962 -1.67839585]
 [-2.293594    3.83963535 -0.22713198]]
[[ 0.          0.          0.        ]
 [-1.526       0.          0.        ]
 [-2.03375551  1.43904842  0.        ]
 [-3.48238364  1.4663484   0.47896477]
 [-3.5869252   2.33224155  1.7311533 ]
 [-2.52933762  3.43105962  1.67839585]
 [-1.29164169  2.91015495  0.9535213 ]]
[[ 0.          0.          0.        ]
 [-1.526       0.          0.        ]
 [-2.03375551  1.43904842  0.        ]
 [-3.48238364  1.4663484   0.47896477]
 [-3.5869252   2.33224